In [ ]:
# First cell - Installation
!pip install flask flask-sqlalchemy flask-socketio pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.6 MB/s eta 0:00:00


In [ ]:
# Second cell - Complete Application Code
from flask import Flask, render_template_string, request, jsonify
from flask_socketio import SocketIO
from flask_sqlalchemy import SQLAlchemy
from pyngrok import ngrok
from datetime import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///consumables_inventory.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = 'your_secret_key'

db = SQLAlchemy(app)
socketio = SocketIO(app)

# Database Model
class Consumable(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), unique=True, nullable=False)
    current_stock = db.Column(db.Integer, nullable=False)
    low_stock_threshold = db.Column(db.Integer, nullable=False)
    unit = db.Column(db.String(50), nullable=False)
    last_restocked = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            'id': self.id,
            'name': self.name,
            'current_stock': self.current_stock,
            'low_stock_threshold': self.low_stock_threshold,
            'unit': self.unit,
            'status': 'Low Stock' if self.current_stock <= self.low_stock_threshold else 'Sufficient',
            'last_restocked': self.last_restocked.strftime('%Y-%m-%d %H:%M:%S')
        }

# Initialize Database
with app.app_context():
    db.create_all()

# HTML Template
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Hospital Consumables Inventory</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        .status-low { background-color: #fff3cd; color: #856404; }
        .status-sufficient { background-color: #d4edda; color: #155724; }
    </style>
</head>
<body>
    <div class="container mt-5">
        <h1 class="text-center mb-4">🏥 Hospital Consumables Inventory</h1>

        <div class="card mb-4">
            <div class="card-header">Add New Consumable</div>
            <div class="card-body">
                <form id="addConsumableForm">
                    <div class="row">
                        <div class="col-md-3">
                            <input type="text" class="form-control" id="name" placeholder="Consumable Name" required>
                        </div>
                        <div class="col-md-2">
                            <input type="number" class="form-control" id="current_stock" placeholder="Current Stock" required>
                        </div>
                        <div class="col-md-2">
                            <input type="number" class="form-control" id="threshold" placeholder="Low Stock Alert" required>
                        </div>
                        <div class="col-md-2">
                            <input type="text" class="form-control" id="unit" placeholder="Unit (pcs, boxes)" required>
                        </div>
                        <div class="col-md-3">
                            <button type="submit" class="btn btn-primary">Add Consumable</button>
                        </div>
                    </div>
                </form>
            </div>
        </div>

        <table class="table table-striped" id="inventoryTable">
            <thead>
                <tr>
                    <th>Consumable</th>
                    <th>Current Stock</th>
                    <th>Low Stock Threshold</th>
                    <th>Unit</th>
                    <th>Last Restocked</th>
                    <th>Status</th>
                    <th>Actions</th>
                </tr>
            </thead>
            <tbody id="inventoryBody"></tbody>
        </table>
    </div>

    <script>
        document.getElementById('addConsumableForm').addEventListener('submit', function(e) {
            e.preventDefault();
            const consumableData = {
                name: document.getElementById('name').value,
                current_stock: parseInt(document.getElementById('current_stock').value),
                threshold: parseInt(document.getElementById('threshold').value),
                unit: document.getElementById('unit').value
            };
            fetch('/add_consumable', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify(consumableData)
            }).then(response => response.json())
              .then(() => location.reload());
        });

        function updateStock(id, change) {
            fetch('/update_stock', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({id, change})
            }).then(response => response.json())
              .then(() => location.reload());
        }

        function restock(id) {
            fetch('/restock', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({id})
            }).then(response => response.json())
              .then(() => location.reload());
        }

        fetch('/get_inventory')
            .then(response => response.json())
            .then(consumables => {
                const inventoryBody = document.getElementById('inventoryBody');
                consumables.forEach(consumable => {
                    const row = document.createElement('tr');
                    row.classList.add(consumable.status === 'Low Stock' ? 'status-low' : 'status-sufficient');
                    row.innerHTML = `
                        <td>${consumable.name}</td>
                        <td>${consumable.current_stock} ${consumable.unit}</td>
                        <td>${consumable.low_stock_threshold} ${consumable.unit}</td>
                        <td>${consumable.unit}</td>
                        <td>${consumable.last_restocked}</td>
                        <td>${consumable.status}</td>
                        <td>
                            <div class="btn-group">
                                <button onclick="updateStock(${consumable.id}, 1)" class="btn btn-sm btn-success">+</button>
                                <button onclick="updateStock(${consumable.id}, -1)" class="btn btn-sm btn-danger">-</button>
                                <button onclick="restock(${consumable.id})" class="btn btn-sm btn-warning">Restock</button>
                            </div>
                        </td>
                    `;
                    inventoryBody.appendChild(row);
                });
            });
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/add_consumable', methods=['POST'])
def add_consumable():
    data = request.get_json()
    new_consumable = Consumable(
        name=data['name'],
        current_stock=data['current_stock'],
        low_stock_threshold=data['threshold'],
        unit=data['unit']
    )
    db.session.add(new_consumable)
    db.session.commit()
    return jsonify({"status": "success"})

@app.route('/update_stock', methods=['POST'])
def update_stock():
    data = request.get_json()
    consumable = Consumable.query.get(data['id'])
    consumable.current_stock += data['change']
    db.session.commit()
    return jsonify({"status": "success"})

@app.route('/restock', methods=['POST'])
def restock():
    data = request.get_json()
    consumable = Consumable.query.get(data['id'])
    consumable.current_stock += 50  # Dummy restock value
    consumable.last_restocked = datetime.utcnow()
    db.session.commit()
    return jsonify({"status": "success"})

@app.route('/get_inventory')
def get_inventory():
    consumables = Consumable.query.all()
    return jsonify([consumable.to_dict() for consumable in consumables])

if __name__ == '__main__':
    ngrok.set_auth_token("2s2cVty2ALzbzNI7q0RCHwLXZY6_85kgGR6VgmaGdio7UAPMh")
    public_url = ngrok.connect(5000).public_url
    print(f" * ngrok tunnel is running at: {public_url}")
    socketio.run(app, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)

 * ngrok tunnel is running at: https://f0c4-34-80-97-50.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:02:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:02:40] "GET /get_inventory HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:02:40] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:03:39] "POST /add_consumable HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:03:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:03:40] "GET /get_inventory HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:04:38] "POST /add_consumable HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:04:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:04:40] "GET /get_